## 8.3 Naive approach
This code shows a first attempt to exchange a shared secret key

In [ ]:
using ImageShow
using Random
using StrangelyDisplayed
using StrangelyQuantum

In the following method, `aliceBits` and any X gates "belong" to the sender. The Measurement gates and `bobBits` "belong" to the receiver. The argument determines how many bits are constructed and transmitted.

In [ ]:
function naive_transmission(key_size::Integer)
    random = Xoshiro()

    # Generate the key by assigning random values to each bit.
    aliceBits = [rand(random, Bool) for _ = 1:key_size]

    simulator = SimpleQuantumExecutionEnvironment()
    # Create a program that involves 1 qubit for every bit in the key
    program = Program(key_size)
    step1 = Step()
    step2 = Step()
    for i = 1:key_size
        # When a bit is true, a Pauli-X gate is applied
        # to the corresponding qubit.
        aliceBits[i] && addGate(step1, X(i))
        # All qubits will be measured in step 2.
        addGate(step2, Measurement(i))
    end

    addStep(program, step1)
    addStep(program, step2)

    result = runProgram(simulator, program)
    qubit = getQubits(result)

    measurement = Vector{Int}(undef, key_size)
    bobBits = Vector{Bool}(undef, key_size)
    for i = 1:key_size
        # Measures the qubit and print the value next to the
        # original value of the corresponding bit
        measurement[i] = measure(qubit[i])
        bobBits[i] = measurement[i] == 1
        println(
            "Alice sent ",
            aliceBits[i] ? '1' : '0',
            " and Bob received ",
            bobBits[i] ? '1' : '0',
        )
    end
    return program
end

In [ ]:
program = naive_transmission(4)

The result above shows that the qubits are reliably transmitted. However an interloper could insert Measurement gates before the Measurement gates provided. The interloper would measure and retransmit the same bits, and security would be lost.

In [ ]:
drawProgram(program)